In [1]:
#TODO download the model
#TODO run object classification with softmax
#TODO make comments more readable later 

In [2]:
import torch
from torchvision import models, transforms
assert(torch.cuda.is_available() == True) # this is important

In [3]:
# To get that progress bar in the jupyter notebook (with VScode and docker)
from ipywidgets import IntProgress 
#conda install -c anaconda ipywidgets

In [4]:
model = models.resnet50(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/light/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [14]:
from PIL import Image
img = Image.open("cat.png")
transform = transforms.Compose([ transforms.Resize(256), transforms.CenterCrop(224), transforms.ToTensor(),
    transforms.Normalize( (0.485, 0.456, 0.406), (0.229,0.224,0.225))]) # ETL

img = transform(img)
img.shape

torch.Size([3, 224, 224])

In [18]:
img_batch = torch.unsqueeze(img,0) # adds that extra dimension in the beginnning
img_batch.shape

torch.Size([1, 3, 224, 224])

In [19]:
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [24]:
# let's do the final part
# at inference time we don't need gradients associated with the tensor
# so we can deactivate it for now

with torch.no_grad():
    outputs = model(img_batch)
probs = torch.nn.functional.softmax(outputs[0],0) # this results in a big vector with about 1000 classes
# ideally one of these classes is the right prediction

torch.Size([1, 1000])

In [36]:
# let's associate index with actual name of class from dictionary
import pandas as pd
categories = pd.read_csv("https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt", header=None)
# categories has dictionary attributes like : { '0': 'fish} for example
categories.shape

(1000, 1)

In [43]:
categories[0] # has all the names

tench


In [35]:
top3 = 3 # top k predictions
prob, class_number = torch.topk(probs, top3) # extract top 3 highest probablities

torch.Size([3])

In [44]:
for i in range(top3):
    probability = prob[i].item()
    class_name = categories[0][int(class_number[i])]
    print( "%{} probability for class  {}".format (int(probability*100), class_name))


%43 probability for class  Persian cat
%25 probability for class  Egyptian cat
%4 probability for class  tiger cat
